In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [3]:
loader = TextLoader("../data/dataengineering.txt", encoding="utf-8")
documents = loader.load()
documents

[Document(metadata={'source': '../data/dataengineering.txt'}, page_content='Agentic AI refers to intelligent systems that can autonomously plan, decide, and execute actions toward specific predefined goals without constant human supervision.\nUnlike traditional software programs, an agent observes its environment, reasons about possible outcomes, and selects the most appropriate action dynamically.\nThe main purpose of agentic systems is reducing manual effort while improving decision-making efficiency in complex workflows.\nThese systems typically include memory components, reasoning engines, and the ability to call external tools or APIs.\nAn AI agent can interact with databases, web services, or enterprise applications to complete assigned objectives.\nPlanning capabilities allow the agent to decompose large tasks into smaller manageable steps before execution.\nReactive agents respond immediately to stimuli, whereas deliberative agents evaluate multiple possibilities before acting.

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)
chunks = splitter.split_documents(documents)
chunks_str = [chunk.page_content for chunk in chunks]
chunks

[Document(metadata={'source': '../data/dataengineering.txt'}, page_content='Agentic AI refers to intelligent systems that can autonomously plan, decide, and execute actions'),
 Document(metadata={'source': '../data/dataengineering.txt'}, page_content='and execute actions toward specific predefined goals without constant human supervision.'),
 Document(metadata={'source': '../data/dataengineering.txt'}, page_content='Unlike traditional software programs, an agent observes its environment, reasons about possible'),
 Document(metadata={'source': '../data/dataengineering.txt'}, page_content='about possible outcomes, and selects the most appropriate action dynamically.'),
 Document(metadata={'source': '../data/dataengineering.txt'}, page_content='The main purpose of agentic systems is reducing manual effort while improving decision-making'),
 Document(metadata={'source': '../data/dataengineering.txt'}, page_content='decision-making efficiency in complex workflows.'),
 Document(metadata={'so